In [1]:
from util import *
from util_data_storage_and_load import *
import numpy as np
from numpy.linalg import inv
from scipy.sparse import csr_matrix, csc_matrix
import json

No dicts found; please check load_dicts...


In [2]:
with open('../temp_files/new_route_dict_journal.json', 'r') as json_file:
    new_route_dict = json.load(json_file)
    
number_of_routes = len(new_route_dict)

In [3]:
# implement GLS method to estimate OD demand matrix
def GLS(x, A, L):
    """
    x: sample matrix, each column is a link flow vector sample; number_of_links * K
    A: path-link incidence matrix
    P: logit route choice probability matrix
    L: dimension of xi
    ----------------
    return: xi
    ----------------
    """
    K = np.size(x, 1)
    S = samp_cov(x)

    #print("rank of S is: \n")
    #print(matrix_rank(S))
    #print("sizes of S are: \n")
    #print(np.size(S, 0))
    #print(np.size(S, 1))

    inv_S = inv(S).real

    A_t = A.transpose()

    Q_ = A_t * inv_S * A
    Q_ = Q_.real
#     Q = adj_PSD(Q_)  # Ensure Q to be PSD
    Q = Q_

    #print("rank of Q is: \n")
    #print(matrix_rank(Q))
    #print("sizes of Q are: \n")
    #print(np.size(Q, 0))
    #print(np.size(Q, 1))

    b = sum([A_t * inv_S * x[:, k] for k in range(K)])
    # print(b[0])
    # assert(1==2)

    model = Model("OD_matrix_estimation")

    xi = []
    for l in range(L):
        xi.append(model.addVar(name='xi_' + str(l)))

    model.update() 

    # Set objective: (K/2) xi' * Q * xi - b' * xi
    obj = 0
    for i in range(L):
        for j in range(L):
            obj += (1.0 / 2) * K * xi[i] * Q[i, j] * xi[j]
    for l in range(L):
        obj += - b[l] * xi[l]
    model.setObjective(obj)

    # Add constraint: xi >= 0
    for l in range(L):
        model.addConstr(xi[l] >= 0)
        #model.addConstr(xi[l] <= 5000)
    #fictitious_OD_list = zload('../temp_files/fictitious_OD_list')
    #for l in fictitious_OD_list:
        #model.addConstr(xi[l] == 0)
    model.update() 

    #model.setParam('OutputFlag', False)
    model.optimize()

    xi_list = []
    for v in model.getVars():
        # print('%s %g' % (v.varName, v.x))
        xi_list.append(v.x)
    # print('Obj: %g' % obj.getValue())
    return xi_list

In [4]:
# load link_route incidence matrix
A = zload('../temp_files/link_route_incidence_matrix_journal.pkz')

# load link counts data
with open('../temp_files/link_day_minute_Apr_dict_journal_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

weekend_Apr_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]
# weekend_Apr_list = [9, 10, 11, 12, 13]

feasible_link_dict = zload('../temp_files/feasible_link_dict_journal.pkz')

link_day_minute_Apr_list = []
for link_idx in [feasible_link_dict[idx] for idx in range(len(feasible_link_dict))]:
    for day in weekend_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['PM_flow_minute'][minute_idx])

x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, len(feasible_link_dict), 1080)
# x = np.matrix.reshape(x, len(feasible_link_dict), 600)

# print(np.size(x,0), np.size(x,1))

x = np.nan_to_num(x)
# print(np.size(x,0), np.size(x,1))

# y = np.array(np.transpose(x))
# y = y[np.all(y != 0, axis=1)]
# x = np.transpose(y)
# x = np.matrix(x)

# print(np.size(x,0), np.size(x,1))
# print(x[:,:2])
# print(np.size(A,0), np.size(A,1))

### If we see an error message like:
#### GurobiError: Objective Q not PSD (diagonal adjustment of "some number" would be required), 
### where "some number" is very small, say, less than 1e-2, then, we can safely use the input flow data.

In [5]:
xi_list = GLS(x, A, number_of_routes)

Optimize a model with 1122 rows, 1122 columns and 1122 nonzeros
Model has 630003 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [4e-01, 6e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 1122 rows and 0 columns
Presolve time: 0.12s
Presolved: 0 rows, 1122 columns, 0 nonzeros
Presolved model has 630003 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 191
 AA' NZ     : 1.814e+04
 Factor NZ  : 1.834e+04 (roughly 1 MByte of memory)
 Factor Ops : 2.341e+06 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -9.95895740e+08 -2.60445307e+05  9.32e+04 6.14e+02  1.00e+06     0s
   1  -1.65575276e+08 -2.63107518e+07  1.66e+04 1.63e+02  2.19e+05     0s
   2  -4.13934362e+06 -2.04592219e+07  9.35e+02 9.20e+00  3.19e+04     0s
   3  -2.75015734e

In [6]:
xi_list

[143.00473287962663,
 299.4185143241775,
 344.9867343336998,
 1552.260481461906,
 372.98453537792744,
 50.79438630411774,
 3.0723719654239145e-10,
 112.66690063420039,
 1.9696253005759823,
 6.139535434967952e-10,
 6.139535411413463e-10,
 6.139536575312593e-10,
 6.139535417101719e-10,
 6.139536527259215e-10,
 1.6214385996291323e-11,
 2.5953978863125856e-11,
 6.139535438797856e-10,
 6.13953662714378e-10,
 6.139542460132029e-10,
 6.139536515929959e-10,
 6.139551433466144e-10,
 6.139536129539948e-10,
 6.139536605628117e-10,
 6.139536575263169e-10,
 6.139541062205107e-10,
 5.2735001857907807e-11,
 1.8792841890698757,
 1.9057778292130245,
 6.139537034013533e-10,
 6.139538437239655e-10,
 6.139622025057858e-10,
 6.139538658758888e-10,
 6.139537172898812e-10,
 13.38717159928933,
 13.66097616001054,
 165.13851055220215,
 12.257166624994952,
 704.0574895378185,
 3.3625079878324725e-11,
 79.51950273963745,
 64.59139356980988,
 38.138255062570366,
 102.25587904034477,
 7.73706241477893e-10,
 7.7370